# Sudoku solver

In [2]:
import opvious
import pandas as pd

client = opvious.Client.from_environment()

In [23]:
def grid(triples):
    """Transforms a list of triples into a natural Sudoku 2d grid."""
    positions = list(range(9))
    return (
        pd.DataFrame(triples, columns=['row', 'column', 'value'])
            .pivot_table(index='row', columns='column', values='value')
            .reindex(positions)
            .reindex(positions, axis=1)
            .applymap(lambda v: str(int(v)) if v == v else '')
    )

## Finding an optimal solution

In [24]:
async def solve(inputs):
    """Returns the triples representing a valid solution for the input triples."""
    response = await client.run_solve(
        formulation_name="sudoku",
        parameters={'input': inputs}
    )
    output = response.outputs.variable('output')
    return list(output.index)

In [25]:
grid(await solve([
    (0, 0, 2),
    (1, 1, 4),
    (2, 2, 6),
]))

column,0,1,2,3,4,5,6,7,8
row,,,,,,,,,
0,2,3,1,9,8,4,7,5,6
1,9,4,7,5,6,3,8,1,2
2,5,8,6,7,1,2,4,3,9
3,4,1,5,8,3,6,9,2,7
4,8,7,2,4,5,9,3,6,1
5,3,6,9,1,2,7,5,4,8
6,6,2,4,3,7,8,1,9,5
7,1,9,8,2,4,5,6,7,3
8,7,5,3,6,9,1,2,8,4


## Identifying mistakes

In [26]:
async def assist(inputs):
    """Returns the smallest set of triples to make the input triples feasible."""
    response = await client.run_solve(
        formulation_name="sudoku",
        parameters={'input': inputs},
        relaxation=opvious.Relaxation.from_constraint_labels(['mask']),
    )
    deficit = response.outputs.variable('mask_deficit')
    return list(deficit.index)

In [27]:
grid(await assist([
    (0, 0, 2),
    (1, 1, 2),
]))

column,0,1,2,3,4,5,6,7,8
row,,,,,,,,,
0,,,,,,,,,
1,,2,,,,,,,
2,,,,,,,,,
3,,,,,,,,,
4,,,,,,,,,
5,,,,,,,,,
6,,,,,,,,,
7,,,,,,,,,
8,,,,,,,,,
